In [1]:
#import things

import cv2
import numpy  as np
from collections import deque
import time



In [2]:
from keras.models import load_model
cnn_model = load_model('emnist_cnn_model.h5')

In [3]:
prediction_list = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r",
                  "s", "t", "u", "v", "w", "x", "y", "z"]

In [4]:
#define things
#engine = pyttsx3.init()
points = deque(maxlen = 1000)
red_upper = (150,150,50)
red_lower = (200,255,150)
blackboard = np.zeros((512,512), np.uint8)

In [5]:
start_time = time.time()


cap = cv2.VideoCapture(0)
points
    
while time.time() < start_time + 20:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    #hsv
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #red
    red = cv2.inRange(hsv, red_upper, red_lower)
    #red median blur
    red_median = cv2.medianBlur(red, 15)
    #find contours
    contours, _ = cv2.findContours(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    try:
        if len(contours) > 0:
            contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
            contour = contour_list[0]
            #find centroid
            M = cv2.moments(contour) #M will be a dict
            centroid_x = int(M["m10"]/M["m00"])
            centroid_y = int(M["m01"]/M["m00"])
            centroid = (centroid_x, centroid_y)
            #append centroid in points
            points.appendleft(centroid)
            #draw circle
            ((x,y), r) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x),int(y)), int(r), (0,255,0), 2)
    except ZeroDivisionError:
        pass
            
    for i in range(1, len(points)):
        cv2.line(frame, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (0,0,0), 2)
        cv2.line(blackboard, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (255,255,255), 8)

    cv2.imshow("frame", frame)
    cv2.imshow("blackboard", blackboard)
    
    if cv2.waitKey(1) &  0xFF == ord("q"):
        break

    
blackboard_median = cv2.medianBlur(blackboard, 15)
alphabet = cv2.resize(blackboard_median, (28,28))
alphabet = np.array(alphabet)
alphabet = alphabet.astype("float32")/255
prediction = cnn_model.predict(alphabet.reshape(28,28))[0]
prediction = np.argmax(prediction)
print(prediction_list[prediction])


#cv2.putText(blackboard, "prediction" + prediction_list[prediction], (20,450), cv2.FONT_HERSHEY_SIMPLEX,
#           0.8, (255,255,255), 2)

#while time.time() > start_time + 21 and time.time() < start_time + 25:
#    cv2.imshow("blackboard", blackboard)


#    if cv2.waitKey(1) & 0xFF == ord("q"):
#        break
    
points = deque(maxlen = 1000)
blackboard = np.zeros((512,512), np.uint8)

cv2.destroyAllWindows()
cap.release()

ValueError: in user code:

    c:\users\prajwal\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1544 predict_function  *
        return step_function(self, iterator)
    c:\users\prajwal\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1527 run_step  *
        outputs = model.predict_step(data)
    c:\users\prajwal\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1500 predict_step  *
        return self(x, training=False)
    c:\users\prajwal\appdata\local\programs\python\python39\lib\site-packages\keras\engine\base_layer.py:989 __call__  *
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    c:\users\prajwal\appdata\local\programs\python\python39\lib\site-packages\keras\engine\input_spec.py:227 assert_input_compatibility  *
        raise ValueError('Input ' + str(input_index) + ' of layer ' +

    ValueError: Input 0 of layer sequential is incompatible w